### Import libraries

In [31]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

### Load Chicken Egg data

In [32]:
data2 = pd.read_csv('data2.csv')
data2.head()

,chicken,egg
0,468491,3581
1,449743,3532
2,436815,3327
3,444523,3255
4,433937,3156


### Run granger test

In [33]:
gc_res = grangercausalitytests(data2, 5)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.2071  , p=0.2772  , df_denom=50, df_num=1
ssr based chi2 test:   chi2=1.2795  , p=0.2580  , df=1
likelihood ratio test: chi2=1.2643  , p=0.2608  , df=1
parameter F test:         F=1.2071  , p=0.2772  , df_denom=50, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.8175  , p=0.0006  , df_denom=47, df_num=2
ssr based chi2 test:   chi2=19.5110 , p=0.0001  , df=2
likelihood ratio test: chi2=16.5676 , p=0.0003  , df=2
parameter F test:         F=8.8175  , p=0.0006  , df_denom=47, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=5.4050  , p=0.0030  , df_denom=44, df_num=3
ssr based chi2 test:   chi2=18.7946 , p=0.0003  , df=3
likelihood ratio test: chi2=16.0003 , p=0.0011  , df=3
parameter F test:         F=5.4050  , p=0.0030  , df_denom=44, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=4.2568  , p=0.0057  , df_d

### Minimum p-value <= 0.05 is best significance 

In [34]:
min_p_value = 0.05
best_lag = 0
for k,v in gc_res.items():
    if v[0]['ssr_ftest'][1] <= min_p_value:
        best_lag = k
        min_p_value = v[0]['ssr_ftest'][1]
if not best_lag:
    raise Exception('No causality between time series')
best_lag    

2

### Find impact
Model-1 is $y_t = a_1 y_{t-1} +...+ a_p y_{t-p} + a_0 $.


In [35]:
# Model-1
gc_res[best_lag][1][0].params

array([7.57596079e-01, 1.14149450e-01, 5.19829118e+04])

Model-2 is $y_t = a_1 y_{t-1} +...+ a_p y_{t-p} + b_1 x_{t-1} +...+ b_p x_{t-p} + a_0$.

In [36]:
# Model-2
gc_res[best_lag][1][1].params

array([ 3.27967397e-01,  4.64590640e-01,  8.92693021e+01, -9.42841738e+01,
        1.05659724e+05])

We average the x term coefficients, $\frac{\sum\nolimits_{i=1}^{p} b_i}{|p|}$, as an impact value from model-2.

In [37]:
np.sum(gc_res[best_lag][1][1].params[best_lag:-1])/np.abs(best_lag)

-2.5074358647527077